In [1]:
import sys

sys.path.append('../')

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from config.regressors import *
from config.models import *
from config.loss_functions import *
from config.transformers import *
from config.validation import *
from config.dann import *

In [6]:
class AutoEncoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        
        # Encoder: [3 → 64 → 32 → 16 → latent_dim]
        self.encoder = nn.Sequential(
            nn.Linear(3, 16),
            nn.SELU(),
            nn.Linear(16, 8),
            nn.SELU(),
            nn.Linear(8, 4),
            nn.SELU(),
            nn.Linear(4, latent_dim)  # No activation
        )
        
        # Decoder: [latent_dim → 16 → 32 → 64 → 3]
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 4),
            nn.SELU(),
            nn.Linear(4, 8),
            nn.SELU(),
            nn.Linear(8, 16),
            nn.SELU(),
            nn.Linear(16, 3)  # No activation
        )
        
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                # LeCun normal initialization for SELU
                nn.init.normal_(m.weight, 0, std=1. / np.sqrt(m.in_features))
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        return self.decoder(self.encoder(x))

In [ ]:
def trainer(bone, latent_dim):
    # ----------------------------
    # 1. Configuration
    # ----------------------------
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Hyperparameters
    batch_size     = 4096
    learning_rate  = 1e-3
    weight_decay   = 1e-4
    max_epochs     = 100
    patience       = 20
    num_workers    = 4  # >0 for parallel loading

    # ----------------------------
    # 2. Prepare Data (CPU‑only)
    # ----------------------------
    # Assume Y is your NumPy array of shape [N, 3*B]
    PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'
    Y = np.load(PATH + 'Y_all.npy')
    Y_bone = Y[:, bone*3:(bone+1)*3]
    Y_bone = Y[:, bone*3:(bone+1)*3]

    Yb_cpu   = torch.as_tensor(Y_bone, dtype=torch.float32)  # stays on CPU

    dataset      = TensorDataset(Yb_cpu, Yb_cpu)
    train_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        persistent_workers=True
    )
    val_loader   = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        persistent_workers=True
    )

    # instantiate with your chosen latent dimension
    model = AutoEncoder(latent_dim).to(device)

    # ----------------------------
    # 4. Loss & Optimizer
    # ----------------------------
    criterion = nn.MSELoss()
    optimizer = optim.Adam(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )

    # ----------------------------
    # 5. Training Loop w/ Early Stopping
    # ----------------------------
    best_val_loss     = float('inf')
    epochs_no_improve = 0

    for epoch in range(1, max_epochs+1):
        # —— Training —— 
        model.train()
        running_loss = 0.0

        for xb_cpu, yb_cpu in train_loader:
            # move batch to MPS (or CUDA) on-the-fly
            xb = xb_cpu.to(device, non_blocking=True)
            yb = yb_cpu.to(device, non_blocking=True)

            optimizer.zero_grad()
            preds = model(xb)              
            loss  = criterion(preds, yb)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # —— Validation ——
        model.eval()
        running_val = 0.0
        with torch.no_grad():
            for xb_cpu, yb_cpu in val_loader:
                xb = xb_cpu.to(device, non_blocking=True)
                yb = yb_cpu.to(device, non_blocking=True)
                running_val += criterion(model(xb), yb).item()
        avg_val_loss = running_val / len(val_loader)

        print(f"Epoch {epoch:02d}: Train MSE = {avg_train_loss:.6f} | Val MSE = {avg_val_loss:.6f}")

        # —— Early Stopping ——
        if avg_val_loss < best_val_loss:
            best_val_loss     = avg_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), f"best_models_cuda/best_model_bone{bone}.pth")
            print("  → New best model saved")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early stopping triggered")
                break

In [ ]:
# PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'

# DATASET = 'guided'
# Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')
# Y = Y.transpose(0, 2, 1).reshape(-1, Y.shape[1])

# DATASET = 'freemoves'
# YY = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')
# YY = YY.transpose(0, 2, 1).reshape(-1, YY.shape[1])

# Y_all = np.vstack([Y, YY])

# np.save(PATH + 'Y_all.npy', Y_all)

# latent_dims = {
#         0: 1, 1: 1, 2: 1,
#         3: 1, 4: 2, 5: 1,
#         6: 1, 7: 2, 8: 1,
#         9: 1, 10: 2, 11: 1,
#         12: 1, 13: 1, 14: 2,
#         15: 1, 16: 1,}

# for bone in range(17):
#     trainer(bone, latent_dims[bone])

Using device: cuda
Epoch 01: Train MSE = 201.669550 | Val MSE = 45.854745
  → New best model saved
Epoch 02: Train MSE = 44.262359 | Val MSE = 42.299581
  → New best model saved
Epoch 03: Train MSE = 39.035336 | Val MSE = 34.545258
  → New best model saved
Epoch 04: Train MSE = 23.843714 | Val MSE = 9.010370
  → New best model saved
Epoch 05: Train MSE = 2.457708 | Val MSE = 0.443657
  → New best model saved
Epoch 06: Train MSE = 0.300545 | Val MSE = 0.237245
  → New best model saved
Epoch 07: Train MSE = 0.228174 | Val MSE = 0.223350
  → New best model saved
Epoch 08: Train MSE = 0.224349 | Val MSE = 0.224297
Epoch 09: Train MSE = 0.224494 | Val MSE = 0.222168
  → New best model saved
Epoch 10: Train MSE = 0.224661 | Val MSE = 0.223163
Epoch 11: Train MSE = 0.224339 | Val MSE = 0.236498
Epoch 12: Train MSE = 0.224620 | Val MSE = 0.224000
Epoch 13: Train MSE = 0.224116 | Val MSE = 0.219629
  → New best model saved
Epoch 14: Train MSE = 0.212893 | Val MSE = 0.200944
  → New best model s

In [8]:
def encode(Y):
    latent_dims = {
        0: 1, 1: 1, 2: 1,
        3: 1, 4: 2, 5: 1,
        6: 1, 7: 2, 8: 1,
        9: 1, 10: 2, 11: 1,
        12: 1, 13: 1, 14: 2,
        15: 1, 16: 1,}
    
    Y_shape = Y.shape
    
    Y_enc = []
    for bone in range(17):
        latent_dim = latent_dims[bone]
        model = AutoEncoder(latent_dim=latent_dim).to('cuda')
        model.load_state_dict(torch.load(f'best_models_cuda/best_model_bone{bone}.pth'))
        
        to_encode = Y[..., 3*bone:3*(bone+1)].reshape(-1,3)
        to_encode_tensor = torch.tensor(
            to_encode,
            dtype=torch.float,
            device='cuda')
        encoded_tensor = model.encoder(to_encode_tensor)
        Y_enc.append(encoded_tensor.cpu().detach().numpy())

    Y_enc = np.hstack(Y_enc)
    Y_enc = Y_enc.reshape(*Y_shape[:-1], 21)
    
    return Y_enc

In [9]:
def decode(Y):
    latent_dims = {
        0: 1, 1: 1, 2: 1,
        3: 1, 4: 2, 5: 1,
        6: 1, 7: 2, 8: 1,
        9: 1, 10: 2, 11: 1,
        12: 1, 13: 1, 14: 2,
        15: 1, 16: 1,}
    
    Y_shape = Y.shape
    
    current = 0
    Y_dec = []
    for bone in range(17):
        latent_dim = latent_dims[bone]
        model = AutoEncoder(latent_dim=latent_dim).to('cuda')

        model.load_state_dict(torch.load(f'best_models_cuda/best_model_bone{bone}.pth'))

        to_decode = Y[..., current:current + latent_dim].reshape(-1, latent_dim)
        current += latent_dim

        to_decode_tensor = torch.tensor(
            to_decode,
            dtype=torch.float,
            device='cuda')
        decoded_tensor = model.decoder(to_decode_tensor)
        Y_dec.append(decoded_tensor.cpu().detach().numpy())
        
    Y_dec = np.hstack(Y_dec)
    Y_dec = Y_dec.reshape(*Y_shape[:-1], 51)

    return Y_dec

## Prediction generation

In [ ]:
from config.transformers import TimeDomainTransformer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

import pyriemann
import pyriemann.regression

from xgboost import XGBRegressor

baseline_kr = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', KernelRidge(
            alpha = 0.01,
            gamma = 0.01,
            kernel='laplacian'))
    ]
)

baseline_knn = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', KNeighborsRegressor(
            n_neighbors = 5))
    ]
)

baseline_rf = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', RandomForestRegressor(
            n_estimators = 50,
            max_depth = 10,
            random_state=42))
    ]
)

baseline_xgb = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', XGBRegressor())
    ]
)

riem_kr = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', KernelRidge(
            alpha = 0.01,
            gamma = 0.01,
            kernel='laplacian'))
    ]
)

riem_knn = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', KNeighborsRegressor(
            n_neighbors = 5))
    ]
)

riem_rf = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', RandomForestRegressor(
            n_estimators = 50,
            max_depth = 10,
            random_state=42))
    ]
)

riem_xgb = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', XGBRegressor())
    ]
)

In [ ]:
model = StackingRegressor(
    estimators = [
        baseline_kr,
        baseline_knn,
        baseline_rf,
        # riem_kr,
        riem_knn,
        # riem_rf
    ],
    end_estimator = KNeighborsRegressor(
        n_neighbors=7,
        p=1
    ),
    n_internal_folds=5
)

PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'

confirmation = input('Have you inserted the correct model name and the expected RMSE?')

model_name = 'prova_stacking'
expected_rmse = '1000'

step_prediction = 50 # step for prediction

# preparing the training data
tw_extractor = TimeWindowTransformer(size = 500, step = step_prediction)
label_extractor = LabelWindowExtractor(size = 500, step = step_prediction)

DATASET = 'guided'
X = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

DATASET = 'freemoves'
XX = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
YY = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

XX = XX[..., -230000:]
YY = YY[..., -230000:]

X = np.vstack([X, XX])
Y = np.vstack([Y, YY])

X_windows = tw_extractor.transform(X)
Y_labels = label_extractor.transform(Y)

# # encoding
# print('Encoding...')
# Y_labels = encode(Y_labels)
# print('Encoded.')

# stacking the sessions 
X_train = X_windows.reshape(-1, *X_windows.shape[2:])
Y_train = Y_labels.reshape(-1, *Y_labels.shape[2:])

# # scaling the target
# targ_scal = StandardScaler()
# Y_train = targ_scal.fit_transform(Y_train)

# training
print('Training...')
model.fit(X_train, Y_train)
print('Trained.')

# # predicting
# DATASET = 'guided'
# X_test = np.load(PATH + f'{DATASET}/{DATASET}_testset_X.npy')
# X_test = X_test.reshape(-1, *X_windows.shape[2:])
# print('Predicting...')
# Y_pred = model.predict(X_test)
# print('Predicted.')

# # scaling back
# Y_pred = targ_scal.inverse_transform(Y_pred)

# # decoding
# print('Decoding...')
# Y_pred = decode(Y_pred)
# print('Decoded.')

# saving
file_name = 'guided_encoding_scaling_meta_50.npy'

np.save(file_name, Y_pred)

Y_pred_guided = np.load('guided_encoding_scaling_meta_50.npy')
Y_pred_freemoves = np.load('dann_steps_50_rmse_10.npy')

Y_pred = np.vstack([Y_pred_guided, Y_pred_freemoves])
Y_pred_df = pd.DataFrame(Y_pred)
Y_pred_df.to_csv('very_last_try.csv', index=False, header=False)

In [13]:
PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'

model_name = 'prova_autoencoders_dann'
expected_rmse = '1000'

step_prediction = 50 # step for prediction

# preparing the training data
tw_extractor = TimeWindowTransformer(size = 500, step = step_prediction)
label_extractor = LabelWindowExtractor(size = 500, step = step_prediction)

DATASET = 'freemoves'

X = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

# DATASET = 'guided'

# X = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
# Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

# DATASET = 'freemoves'

# XX = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
# YY = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

# XX = XX[..., -230000:]
# YY = YY[..., -230000:]

# X = np.vstack([X, XX])
# Y = np.vstack([Y, YY])

X_windows = tw_extractor.transform(X)
Y_labels = label_extractor.transform(Y)
session_ids = np.concatenate([
    np.full(X_windows[i].shape[0], i) for i in range(len(X_windows))
])

# Instantiate DANNRegressor
dann = DANNModel(
    lambda_grl=1.0, 
    num_domains=5, 
    output_dim=21
)

# temp_dann = TemporalDANNModel(
#     lambda_grl=1.0,
#     num_domains=5,
#     output_dim=51,
#     in_channels=8 * (2 if hasattr(X_transformed, 'ndim') and X_transformed.ndim == 4 else 1), # Adjust in_channels based on potential DeltaFeatureTransformer output
#     mode=1  # Or your desired mode
# )

model = DANNRegressor(
    model=dann,
    dataset_class=DANNWindowTensor,
    gamma_entropy=0.1,
    learning_rate=0.001,
    batch_size=128,
    max_epochs=50,
    session_ids=session_ids
)

# encoding
Y_labels_enc = encode(Y_labels)

# stacking the sessions 
X_train = X_windows.reshape(-1, *X_windows.shape[2:])
Y_train = Y_labels_enc.reshape(-1, *Y_labels_enc.shape[2:])

# training
model.fit(X_train, Y_train)

Epoch 01 | Train RMSE: 32.2118 | Train NMSE: 4.8147
           λ = 0.100
Epoch 02 | Train RMSE: 24.1203 | Train NMSE: 2.6996
           λ = 0.200
Epoch 03 | Train RMSE: 17.2390 | Train NMSE: 1.3790
           λ = 0.300
Epoch 04 | Train RMSE: 13.2059 | Train NMSE: 0.8092
           λ = 0.400
Epoch 05 | Train RMSE: 11.5614 | Train NMSE: 0.6202
           λ = 0.500
Epoch 06 | Train RMSE: 10.7124 | Train NMSE: 0.5325
           λ = 0.600
Epoch 07 | Train RMSE: 10.4088 | Train NMSE: 0.5027
           λ = 0.700
Epoch 08 | Train RMSE: 10.2001 | Train NMSE: 0.4828
           λ = 0.800
Epoch 09 | Train RMSE: 10.0450 | Train NMSE: 0.4682
           λ = 0.900
Epoch 10 | Train RMSE: 9.9303 | Train NMSE: 0.4576
           λ = 1.000
Epoch 11 | Train RMSE: 9.7924 | Train NMSE: 0.4450
           λ = 1.000
Epoch 12 | Train RMSE: 9.6875 | Train NMSE: 0.4355
           λ = 1.000
Epoch 13 | Train RMSE: 9.5766 | Train NMSE: 0.4256
           λ = 1.000
Epoch 14 | Train RMSE: 9.5059 | Train NMSE: 0.4193
    

DANNRegressor(dataset_class=<class 'config.dann.DANNWindowTensor'>,
              gamma_entropy=0.1,
              model=DANNModel(
  (feature_extractor): ConvFeatureExtractor(
    (net): Sequential(
      (0): Conv1d(8, 128, kernel_size=(16,), stride=(1,), padding=(8,))
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv1d(128, 16, kernel_size=...
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=64, out_features=32, bias=True)
      (5): ReLU()
      (6): Dropout(p=0.2, inplace=False)
      (7): Linear(in_features=32, out_features=5, bias=True)
    )
  )
),
              session_ids=array([0, 0, 0, ..., 4, 4, 4], shape=(26955,)))

In [14]:
# predicting
DATASET = 'freemoves'
X_test = np.load(PATH + f'{DATASET}/{DATASET}_testset_X.npy')
X_test = X_test.reshape(-1, *X_windows.shape[2:])
Y_pred_enc = model.predict(X_test)

# decoding
Y_pred_dec = decode(Y_pred_enc)

# saving
file_name = 'prova_encoded_decoded_dann_0.001.npy'

np.save(file_name, Y_pred_dec)